<h2><center> CT Manager - Método 1 - Carregamento das Imagens diretamente na árvore </center></h2>

### Grupo

Artur, David, Felipe, Felipe, Jacqueline, Leonardo, Lucas, Matheus, Samira, Vittória

#### Bibliotecas necessárias para execução do programa

In [1]:
# pip install nibabel
# pip install bplustree
# pip install uuid
# pip install pandas
# pip install matplotlib

#### Importando bibliotecas necessárias

In [34]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from bplustree import BPlusTree as bpt
from bplustree import UUIDSerializer
import uuid
import os
import pickle
import uuid
import os
import json
import gc
from time import sleep
import ipywidgets as widgets

#### Função para carregamento da árvore

In [3]:
def initializeTree():
    return bpt('tmp/bplustree.db', serializer=UUIDSerializer(), key_size=16)

In [4]:
imgs = []
secs = []

tree = initializeTree()
tree.checkpoint()

In [5]:
def dropFromRecord(i):
    if (i < len(ims)):
        ims.pop(i)

#### Função para exibição de imagens

In [37]:
def section_widget(sec1=0, sec2=0):
    length = len(imgs)
    
    if (length == 0):
        print('Nenhuma imagem carregada')
        return
    
    plt.figure(figsize = (15,5))
    
    if (length == 1):
        plt.imshow(imgs[0][:,:,sec1])
    else:
        _, axarr = plt.subplots(1,2, figsize = (15,5))

        axarr[0].imshow(imgs[0][:,:,sec1])
        axarr[1].imshow(imgs[1][:,:,sec2])

In [35]:
def showImsAndWidgets():
    if (len(imgs)==0):
        print('Nenhuma imagem carregada')
    elif (len(imgs)==1):
        shp0 = np.shape(imgs[0])
        widgets.interact(section_widget, sec1=(0,shp0[2]))
    else:
        shp0 = np.shape(imgs[0])
        shp1 = np.shape(imgs[1])
        widgets.interact(section_widget, sec1=(0,shp0[2]), sec2=(0,shp1[2]))

#### Apenas 2 imagens devem ficar na memória principal (conforme especificações)
#####    - A função abaixo garante com que essa especificação seja cumprida

In [9]:
# Função addImageToRecord adaptada para árvore B+
def addImageToRecord(img_name, tree, keys):
    
    if len(keys) == 0:
        print('Não há chaves salvas!')
        return
    
    if img_name not in keys:
        print('Imagem não está no banco de dados!')
        return

    im = pickle.loads(tree.get(uuid.UUID(keys.get(img_name)))).get_fdata()
    imgs.append(im)
    secs.append(0)
    
    if(len(imgs) > 2):
        imgs.pop(0)
        secs.pop(0)

#### Função para seleção das seções de cada imagem

#### Função para carregamento das imagens na árvore (ATENÇÃO: a biblioteca utilizada limita a operação de inserção a cada 11gb, portanto, a inserção de 20gb deve ser feita em duas etapas, com cada conjunto de imagens tendo no máximo 11gb)

In [11]:
def chargeTree(img_folder, keys, tree):
    
    img_list = os.listdir(img_folder)
    img_list.sort()
    
    # img_list = img_list[img_list.index('380.nii'):]
    imgs_charge = {}

    n = 0
    
    for image in img_list:
        print(image)
        id = uuid.uuid1()
        imgs_charge[id] = pickle.dumps(nib.load(img_folder + '/' + image))
        keys[image] = str(id)
    
    tree.batch_insert(iter(imgs_charge.items()))
    tree.checkpoint()
        

#### Utilize essa função para salvar as chaves em disco

In [12]:
def saveKeys(keys):
    with open('keys.json', 'w') as js:
        json.dump(keys, js)

#### Utilize essa função caso queira renomear os arquivos na pasta

In [14]:
def rename_files(folder):
    lista = os.listdir(folder)
    n = 212
    
    for i in lista:
        os.rename(folder + '/' + i, folder + '/' + f'{n}.nii')
        n+=1


#### Utilize essa função para carregar na memória principal as chaves

In [15]:
def chargeKeys():
    keys = {}
    with open('keys.json', 'rb') as js:
            return json.load(js)

In [16]:
keys = {}

In [18]:
%%time
# Carregando imagens teste para a árvore B+
tree.checkpoint()
chargeTree('CTS1', keys, tree)

100.nii
101.nii
102.nii
103.nii
104.nii
105.nii
106.nii
107.nii
108.nii
109.nii
110.nii
111.nii
112.nii
113.nii
114.nii
115.nii
116.nii
117.nii
118.nii
119.nii
120.nii
121.nii
122.nii
123.nii
124.nii
125.nii
126.nii
127.nii
128.nii
129.nii
130.nii
131.nii
132.nii
133.nii
134.nii
135.nii
136.nii
137.nii
138.nii
139.nii
140.nii
141.nii
142.nii
143.nii
144.nii
145.nii
146.nii
147.nii
148.nii
149.nii
150.nii
151.nii
152.nii
153.nii
154.nii
155.nii
156.nii
157.nii
158.nii
159.nii
160.nii
161.nii
162.nii
163.nii
164.nii
165.nii
166.nii
167.nii
168.nii
169.nii
170.nii
171.nii
172.nii
173.nii
174.nii
175.nii
176.nii
177.nii
178.nii
179.nii
180.nii
181.nii
182.nii
183.nii
184.nii
185.nii
186.nii
187.nii
188.nii
189.nii
190.nii
191.nii
192.nii
193.nii
194.nii
195.nii
196.nii
197.nii
198.nii
199.nii
200.nii
201.nii
202.nii
203.nii
204.nii
205.nii
206.nii
207.nii
208.nii
209.nii
210.nii
211.nii
CPU times: total: 172 ms
Wall time: 175 ms


In [19]:
saveKeys(keys)

In [20]:
tree.checkpoint()

In [32]:
# Carregando duas imagens para o programa
addImageToRecord('100.nii', tree, keys)
addImageToRecord('200.nii', tree, keys)

In [38]:
# Exibindo imagens carregadas
showImsAndWidgets()

interactive(children=(IntSlider(value=0, description='sec1', max=195), IntSlider(value=0, description='sec2', …

In [ ]:
tree.close()